In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets, transforms

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'

In [3]:
BATCH_SIZE = 16
EPOCHS = 10
LR = 1e-3

In [4]:
trans = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),
])

In [5]:
from torch.utils.data import DataLoader

# 下载数据集
train_set = datasets.MNIST('./data', train=True, download=True, transform=trans)
test_set = datasets.MNIST('./data', train=False, download=True, transform=trans)

# 加载数据集
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)

In [6]:
with open('./data/MNIST/raw/train-images-idx3-ubyte', 'rb') as f:
    train_images = f.read()

In [7]:
train_image1 = [int(str(i).encode('ascii'), 16) for i in train_images[16:16+784]]

In [8]:
import cv2
import numpy as np

train_image1_np = np.array(train_image1, dtype=np.uint16).reshape(28, 28, 1)
cv2.imwrite('train_image1.jpg', train_image1_np)

[ WARN:0@0.005] global loadsave.cpp:1063 imwrite_ Unsupported depth image for selected encoder is fallbacked to CV_8U.


True

In [9]:
class Digit(nn.Module):
    def __init__(self):
        super(Digit, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=3)
        self.fc1 = nn.Linear(in_features=20*10*10, out_features=500)
        self.fc2 = nn.Linear(in_features=500, out_features=10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        input_size = x.size(0) # batch size
        x = self.conv1(x) # batch*1*28*28 => batch*10*24*24(28-5+1) # (输入大小 - 卷积核大小 + 1)
        x = F.relu(x) # batch*10*24*24
        x = F.max_pool2d(x, kernel_size=2, stride=2) # batch*10*24*24 => batch*10*12*12 # (输入大小 - 池化核大小) / 步长 + 1

        x = self.conv2(x) # batch*10*12*12 => batch*20*10*10
        x = F.relu(x)

        x = x.view(input_size, -1) # flatten (-1表示自动计算维度): batch*20*10*10 => batch*2000

        x = self.fc1(x) # batch*2000 => batch*500
        x = F.relu(x)

        x = self.fc2(x) # batch*500 => batch*10

        output = F.log_softmax(x, dim=1)

        return output


In [10]:
model = Digit().to(device)
optimizer = optim.Adam(model.parameters(), lr=LR)

In [11]:
def train_model(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
                data, target = data.to(device), target.to(device)

                optimizer.zero_grad()

                output = model(data)
                loss = F.cross_entropy(output, target)

                loss.backward()
                optimizer.step()

                if batch_idx % 3000 == 0:
                    print(f"Train Epoch: {epoch} \t Loss: {loss.item():.6f}")


In [12]:
def test_model(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.inference_mode():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target).item() # 将一批的损失相加
            pred = output.argmax(dim=1, keepdim=True) # 找到概率最大的下标
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {100. * correct / len(test_loader.dataset):.0f}%\n')

In [13]:
for epoch in range(1, EPOCHS + 1):
    train_model(model, device, train_loader, optimizer, epoch)
    test_model(model, device, test_loader)

Train Epoch: 1 	 Loss: 2.299899
Train Epoch: 1 	 Loss: 0.021098

Test set: Average loss: 0.0031, Accuracy: 98%

Train Epoch: 2 	 Loss: 0.002281
Train Epoch: 2 	 Loss: 0.000654

Test set: Average loss: 0.0022, Accuracy: 99%

Train Epoch: 3 	 Loss: 0.039515
Train Epoch: 3 	 Loss: 0.000204

Test set: Average loss: 0.0026, Accuracy: 99%

Train Epoch: 4 	 Loss: 0.003302
Train Epoch: 4 	 Loss: 0.000022

Test set: Average loss: 0.0025, Accuracy: 99%

Train Epoch: 5 	 Loss: 0.000461
Train Epoch: 5 	 Loss: 0.000140

Test set: Average loss: 0.0041, Accuracy: 99%

Train Epoch: 6 	 Loss: 0.013551
Train Epoch: 6 	 Loss: 0.003990

Test set: Average loss: 0.0038, Accuracy: 99%

Train Epoch: 7 	 Loss: 0.000042
Train Epoch: 7 	 Loss: 0.000306

Test set: Average loss: 0.0026, Accuracy: 99%

Train Epoch: 8 	 Loss: 0.000001
Train Epoch: 8 	 Loss: 0.000006

Test set: Average loss: 0.0035, Accuracy: 99%

Train Epoch: 9 	 Loss: 0.000010
Train Epoch: 9 	 Loss: 0.000002

Test set: Average loss: 0.0043, Accurac